In [2]:
from geographiclib.geodesic import Geodesic

# Ellipsoïde WGS84
geod = Geodesic.WGS84
lat1, lon1, lat2, lon2 = 3.5, 63.5,-25.0, 70.0
# Calcul avec dérivées partielles
result = geod.Inverse(lat1, lon1, lat2, lon2,
                     Geodesic.STANDARD | Geodesic.GEODESICSCALE | Geodesic.REDUCEDLENGTH)

# Récupération des quantités
s12 = result['s12']      # distance
alpha1 = result['azi1']  # azimut départ
alpha2 = result['azi2']  # azimut arrivée
m12 = result['m12']      # longueur réduite
M12 = result['M12']      # échelle géodésique

# Les dérivées partielles se calculent alors avec ces valeurs

In [3]:
M12

0.8736658223535522

In [17]:
import math
from geographiclib.geodesic import Geodesic

def theoretical_derivatives(lat1, lon1, lat2, lon2):

    geod = Geodesic.WGS84

    result = geod.Inverse(
        lat1, lon1, lat2, lon2,
        Geodesic.STANDARD | Geodesic.GEODESICSCALE | Geodesic.REDUCEDLENGTH
    )

    # Extraction des résultats géodésiques
    s12 = result['s12']
    azi1 = result['azi1']
    azi2 = result['azi2']
    m12 = result['m12']
    M12 = result['M12']
    M21 = result['M21']

    # Conversion degrés → radians
    lat2_rad = math.radians(lat2)
    azi2_rad = math.radians(azi2)

    # Paramètres ellipsoïdaux
    a = geod.a
    f = geod.f
    e2 = f * (2 - f)

    # Composantes trigonométriques
    sin_lat2 = math.sin(lat2_rad)
    cos_lat2 = math.cos(lat2_rad)
    sin_azi2 = math.sin(azi2_rad)
    cos_azi2 = math.cos(azi2_rad)

    # Rayons de courbure au point B
    nu2 = 1 - e2 * sin_lat2**2
    N2 = a / math.sqrt(nu2)                    # Rayon de courbure normal au point 2
    M2 = a * (1 - e2) / (nu2**1.5)             # Rayon de courbure méridien au point 2

    # === DÉRIVÉES DE LA DISTANCE s12 ===
    # Basées sur la géométrie différentielle de l'ellipsoïde
    # Ces formules sont directes et bien établies
    ds12_dlat2 = M2 * cos_azi2 /57.29578    # ∂s₁₂/∂φ₂ (composante méridienne)
    ds12_dlon2 = N2 * cos_lat2 * sin_azi2 /57.29578    # ∂s₁₂/∂λ₂ (composante parallèle)

    return {
        's12': s12,
        'azi1': azi1,
        'azi2': azi2,
        'm12': m12,
        'M12': M12,
        'M21': M21,
        'ds12_dlat2': ds12_dlat2,
        'ds12_dlon2': ds12_dlon2,
    }


def validate_with_numerical(lat1, lon1, lat2, lon2, epsilon=1e-6):
    """
    Validation par différentiation numérique
    """
    geod = Geodesic.WGS84

    # Référence
    ref = geod.Inverse(lat1, lon1, lat2, lon2, Geodesic.STANDARD)
    azi2_ref = ref['azi2']
    s12_ref = ref['s12']

    # Perturbations
    lat_plus = geod.Inverse(lat1, lon1, lat2+epsilon, lon2, Geodesic.STANDARD)
    lon_plus = geod.Inverse(lat1, lon1, lat2, lon2+epsilon, Geodesic.STANDARD)

    # Dérivées numériques
    dalpha2_dlat2_num = (lat_plus['azi2'] - azi2_ref) / epsilon
    dalpha2_dlon2_num = (lon_plus['azi2'] - azi2_ref) / epsilon
    ds12_dlat2_num = (lat_plus['s12'] - s12_ref) / epsilon
    ds12_dlon2_num = (lon_plus['s12'] - s12_ref) / epsilon

    return {
        'dalpha2_dlat2_num': dalpha2_dlat2_num,
        'dalpha2_dlon2_num': dalpha2_dlon2_num,
        'ds12_dlat2_num': ds12_dlat2_num,
        'ds12_dlon2_num': ds12_dlon2_num
    }

def geodesic_derivatives_alternative(lat1, lon1, lat2, lon2):
    """
    Méthode alternative utilisant les dérivées directes des fonctions
    de GeographicLib pour comparaison
    """
    geod = Geodesic.WGS84

    eps = 0.0001

    ref = geod.Inverse(lat1, lon1, lat2, lon2,
                      Geodesic.STANDARD | Geodesic.GEODESICSCALE | Geodesic.REDUCEDLENGTH)

    lat_plus = geod.Inverse(lat1, lon1, lat2 + eps, lon2, Geodesic.STANDARD)
    lat_minus = geod.Inverse(lat1, lon1, lat2 , lon2, Geodesic.STANDARD)

    lon_plus = geod.Inverse(lat1, lon1, lat2, lon2 + eps, Geodesic.STANDARD)
    lon_minus = geod.Inverse(lat1, lon1, lat2, lon2 , Geodesic.STANDARD)

    # Différences centrées (ordre 2)
    dalpha2_dlat2 = (lat_plus['azi2'] - lat_minus['azi2']) / ( eps)
    dalpha2_dlon2 = (lon_plus['azi2'] - lon_minus['azi2']) / (eps)

    ds12_dlat2 = (lat_plus['s12'] - lat_minus['s12']) / ( eps)
    ds12_dlon2 = (lon_plus['s12'] - lon_minus['s12']) / (eps)

    return {
        'dalpha2_dlat2_deg': dalpha2_dlat2,
        'dalpha2_dlon2_deg': dalpha2_dlon2,
        'ds12_dlat2': ds12_dlat2,
        'ds12_dlon2': ds12_dlon2,
        'method': 'differences_centrees'
    }


if __name__ == "__main__":
    # Coordonnées d'exemple
    #lat1, lon1, lat2, lon2 = 3.5, 63.5, -25.0, 70.0

    # Méthode analytique corrigée
    exact = theoretical_derivatives(lat1, lon1, lat2, lon2)

    # Validation numérique simple
    numerical = validate_with_numerical(lat1, lon1, lat2, lon2)

    # Méthode alternative (différences centrées)
    alternative = geodesic_derivatives_alternative(lat1, lon1, lat2, lon2)

    print("=== RÉSULTATS AVEC FORMULES ANALYTIQUES CORRIGÉES ===")
    print(f"Distance s₁₂: {exact['s12']/1000:.3f} km")
    print(f"Azimut initial α₁: {exact['azi1']:.6f}°")
    print(f"Azimut final α₂: {exact['azi2']:.6f}°")
    print(f"Longueur réduite m₁₂: {exact['m12']/1000:.3f} km")
    print(f"Échelle géodésique M₁₂: {exact['M12']:.6f}")
    print(f"Échelle géodésique M₂₁: {exact['M21']:.6f}")

    print("\n=== DÉRIVÉES DE LA DISTANCE s₁₂ ===")
    print(f"Analytique  - ∂s₁₂/∂φ₂ = {exact['ds12_dlat2']/1000:.1f} km/°")
    print(f"Numérique   - ∂s₁₂/∂φ₂ = {numerical['ds12_dlat2_num']/1000:.1f} km/°")
    print(f"Centré      - ∂s₁₂/∂φ₂ = {alternative['ds12_dlat2']/1000:.1f} km/°")

    print(f"\nAnalytique  - ∂s₁₂/∂λ₂ = {exact['ds12_dlon2']/1000:.1f} km/°")
    print(f"Numérique   - ∂s₁₂/∂λ₂ = {numerical['ds12_dlon2_num']/1000:.1f} km/°")
    print(f"Centré      - ∂s₁₂/∂λ₂ = {alternative['ds12_dlon2']/1000:.1f} km/°")

=== RÉSULTATS AVEC FORMULES ANALYTIQUES CORRIGÉES ===
Distance s₁₂: 3230.438 km
Azimut initial α₁: 167.779569°
Azimut final α₂: 166.527211°
Longueur réduite m₁₂: 3093.259 km
Échelle géodésique M₁₂: 0.873666
Échelle géodésique M₂₁: 0.873766

=== DÉRIVÉES DE LA DISTANCE s₁₂ ===
Analytique  - ∂s₁₂/∂φ₂ = -107.7 km/°
Numérique   - ∂s₁₂/∂φ₂ = -107.7 km/°
Centré      - ∂s₁₂/∂φ₂ = -107.7 km/°

Analytique  - ∂s₁₂/∂λ₂ = 23.5 km/°
Numérique   - ∂s₁₂/∂λ₂ = 23.5 km/°
Centré      - ∂s₁₂/∂λ₂ = 23.5 km/°


In [16]:
def compare_results(lat1, lon1, lat2, lon2):
    """
    Compare les résultats théoriques et numériques
    """
    print("="*60)
    print(f"VALIDATION GÉODÉSIQUE")
    print(f"Point A: ({lat1:.1f}°, {lon1:.1f}°)")
    print(f"Point B: ({lat2:.1f}°, {lon2:.1f}°)")
    print("="*60)

    # Calculs théoriques
    print("\n📐 CALCULS THÉORIQUES (Équation 27)")
    print("-" * 40)
    theo = theoretical_derivatives(lat1, lon1, lat2, lon2)

    # Calculs numériques
    print("\n🔢 CALCULS NUMÉRIQUES (Différences finies)")
    print("-" * 40)
    num = validate_with_numerical(lat1, lon1, lat2, lon2)

    print(f"∂s/∂lat2 (numérique) = {num['ds12_dlat2_num']:.6f} m/°")
    print(f"∂s/∂lon2 (numérique) = {num['ds12_dlon2_num']:.6f} m/°")

    # Comparaison
    print("\n📊 COMPARAISON")
    print("-" * 40)

    # Dérivées de distance
    err_lat = abs(theo['ds12_dlat2'] - num['ds12_dlat2_num'])
    err_lon = abs(theo['ds12_dlon2'] - num['ds12_dlon2_num'])

    rel_err_lat = err_lat / abs(num['ds12_dlat2_num']) * 100 if num['ds12_dlat2_num'] != 0 else 0
    rel_err_lon = err_lon / abs(num['ds12_dlon2_num']) * 100 if num['ds12_dlon2_num'] != 0 else 0

    print(f"∂s/∂lat2:")
    print(f"  Théorique: {theo['ds12_dlat2']:12.6f} m/°")
    print(f"  Numérique: {num['ds12_dlat2_num']:12.6f} m/°")
    print(f"  Erreur:    {err_lat:12.6f} m/° ({rel_err_lat:.3f}%)")

    print(f"\n∂s/∂lon2:")
    print(f"  Théorique: {theo['ds12_dlon2']:12.6f} m/°")
    print(f"  Numérique: {num['ds12_dlon2_num']:12.6f} m/°")
    print(f"  Erreur:    {err_lon:12.6f} m/° ({rel_err_lon:.3f}%)")

    # Validation
    tolerance = 10e-3  # 10 mm/degré
    success_lat = err_lat < tolerance
    success_lon = err_lon < tolerance

    print(f"\n✅ VALIDATION (tolérance: {tolerance} m/°)")
    print("-" * 40)
    print(f"∂s/∂lat2: {'✓ VALIDÉ' if success_lat else '✗ ÉCHEC'}")
    print(f"∂s/∂lon2: {'✓ VALIDÉ' if success_lon else '✗ ÉCHEC'}")

    if success_lat and success_lon:
        print(f"\n🎉 SUCCÈS: L'équation (27) est validée!")
    else:
        print(f"\n⚠️  ATTENTION: Écarts significatifs détectés")

    return theo, num

# Test avec les coordonnées données
if __name__ == "__main__":
    # Paramètres de test
    lat1, lon1, lat2, lon2 = 3.5, 63.5, -25.0, 70.0

    # Validation principale
    theo, num = compare_results(lat1, lon1, lat2, lon2)

    # Tests supplémentaires avec différentes configurations
    print("\n" + "="*60)
    print("TESTS SUPPLÉMENTAIRES")
    print("="*60)

    test_cases = [
        ("Courte distance", (48.8566, 2.3522, 48.606, 2.76)),  # Paris
        ("Longue distance", (40.7128, -74.0060, -33.9249, 18.4241)),  # NY-Cape Town
        ("Cas polaire", (89.0, 0.0, 88.0, 180.0)),  # Près du pôle
    ]

    for name, coords in test_cases:
        print(f"\n{name}:")
        try:
            theo_test, num_test = compare_results(*coords)
            print("✓ Test réussi")
        except Exception as e:
            print(f"✗ Erreur: {e}")

VALIDATION GÉODÉSIQUE
Point A: (3.5°, 63.5°)
Point B: (-25.0°, 70.0°)

📐 CALCULS THÉORIQUES (Équation 27)
----------------------------------------

🔢 CALCULS NUMÉRIQUES (Différences finies)
----------------------------------------
∂s/∂lat2 (numérique) = -107724.491972 m/°
∂s/∂lon2 (numérique) = 23519.711103 m/°

📊 COMPARAISON
----------------------------------------
∂s/∂lat2:
  Théorique: -107724.490541 m/°
  Numérique: -107724.491972 m/°
  Erreur:        0.001432 m/° (0.000%)

∂s/∂lon2:
  Théorique: 23519.709166 m/°
  Numérique: 23519.711103 m/°
  Erreur:        0.001938 m/° (0.000%)

✅ VALIDATION (tolérance: 0.01 m/°)
----------------------------------------
∂s/∂lat2: ✓ VALIDÉ
∂s/∂lon2: ✓ VALIDÉ

🎉 SUCCÈS: L'équation (27) est validée!

TESTS SUPPLÉMENTAIRES

Courte distance:
VALIDATION GÉODÉSIQUE
Point A: (48.9°, 2.4°)
Point B: (48.6°, 2.8°)

📐 CALCULS THÉORIQUES (Équation 27)
----------------------------------------

🔢 CALCULS NUMÉRIQUES (Différences finies)
------------------------

In [6]:
a = geod.a
f = geod.f
b = a-f*a
print(b)
ds12_dlon2 = (a*math.cos( math.radians(lat2)))
print(ds12_dlon2*10**-3)

ds12_dlat2 = math.sqrt(a**2 * math.sin( math.radians(lat2))**2+ b**2  * math.cos( math.radians(lat2)**2))
print(ds12_dlat2*10**-3)

6356752.314245179
5780.555229886578
6851.5681929614475


In [7]:
e2 = f*(2-f)
ep2 = (e2)/(1-e2)
w=1/math.sqrt(1+ep2*math.cos(math.radians(lat2))**2)
rho=(a*w**3)/(1-f)
mu=(a*w**3)/(1-f)
R=mu*math.cos(lat2)

ds12_dlat2 = rho*math.cos(math.radians(alpha2))
print(ds12_dlat2)


-6172158.710635497


In [8]:
a*(1-e2)/w**3

6388119.471791075

In [9]:
math.sin(math.radians(lat2))

-0.42261826174069944

In [10]:
rho = a*(1-e2)**2/((w**3 * 1-e2*math.sin(math.radians(lat2))**2)**1.5)

In [11]:
rho

6383221.591901079

In [12]:
import numpy as np
N2 = a / np.sqrt(1-e2*math.sin(math.radians(lat2))**2)
N2
M2 = a*(1-e2) / np.power(1-e2*math.sin(math.radians(lat2))**2,1.5)
M2

6346818.85872043

In [13]:
(N2*math.cos(math.radians(lat2))*math.sin(math.radians(alpha2)))/57.29578

23519.709165761185

In [14]:
M2*np.cos(np.radians(lat2))/57.29578

100394.32842295522